# Analyze the effect of clustering on the speed data

<b>Intuition</b> : Classification is not possible to classify the speed range between 3.0mph-7.0mph as some people might be walking or running at intermittent speeds such as 4.0-6.0mph.

Therefore in this I have used different clustering algorithms. The challenge associated with this model is that we cannot test it against metrics such as homogeniety score or sihouette coeffecient. So a step in hyperparameter tuning involves using a RandomForestRegressor to rank the models based on the precision observed in the model.

## Algorithms used :
<ol>
    <li>KMeans</li>
    <li>DbScan</li>
    <li>Affinity Propagation</li>
</ol>

# Import Libraries

In [1]:
try:
    rel('DataBuilder')
    rel('MLData')
except:
    print('importing for the first time')
from DataBuilder import DataBuilder
import ClassEvaluator
import numpy as np
import importlib,sys 
from IPython.display import clear_output

importing for the first time


In [2]:
def rel(name):
    importlib.reload(sys.modules[name])

# Build Data 

In [3]:
dataBuilder = DataBuilder(heuristic=False, applyFilter=False)

100%|██████████| 3/3 [00:20<00:00,  6.97s/it]


## KMeans

In [26]:
X_Train, y_Train, X_Test, y_Test = dataBuilder.confs['data1']['conf5']

train_inds = np.where((y_Train<4.2)&(y_Train>6))
test_inds = np.where((y_Test<4.2)&(y_Test>6))

X_Train=X_Train[train_inds]
y_Train=y_Train[train_inds]
X_Test=X_Test[test_inds]
y_Test=y_Test[test_inds]

In [30]:
X_Train.shape

(0,)

In [27]:
# Try kmeans for different number of clusters - 2  
def cluster_labels(arr, criteria):
    split_crirteria = [[[3,5],[5,7]],
                       [[3,4],[4,7]],
                       [[3,6],[6,7]],
                       [[3,4], [4,5] ,[5,7]],
                       [[3,3],[4,4],[5,5],[6,6],[7,7]]]


    label_no = 0
    split = split_crirteria[criteria]
    n_clusters = len(split)
    # Round all the clusters
    newArr = np.array([int(np.round(speed)) for speed in arr])
    # Modify labels of train and test according to the split criteria
    for subSplit in split:
        minim, maxim = subSplit
        indices = np.where((newArr==minim)|(newArr==maxim))
        newArr[indices] = label_no
        newArr = np.array([label_no if speed>=minim and speed<=maxim else speed  
                                 for speed in newArr])
        label_no+=1
        # fit and train the clustering algorithm with appropriate grid search criteria
    return(newArr.ravel())


In [28]:
import matplotlib.pyplot as plt
from collections import Counter
y_test_conv = cluster_labels(y_Test, 1)
y_train_conv  = cluster_labels(y_Train, 1)

In [29]:
from sklearn.cluster import KMeans
from sklearn.metrics import *
km = KMeans(n_clusters=2, max_iter=20000)
km.fit(X_Train)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [8]:
adjusted_rand_score(y_train_conv.ravel(),km.labels_)
#accuracy_score(y_train_conv,km.labels_)

0.00013159829713469155

In [9]:
np.hstack((y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]])

In [10]:
import numpy as np
a=np.array([1,2,3,4,5,6,78])

In [11]:
np.where((a==1)|(a==2))

(array([0, 1]),)

In [12]:
inds = [0,1,2]

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))


              precision    recall  f1-score   support

           0       0.50      0.49      0.50      1854
           1       0.48      0.49      0.48      1759

    accuracy                           0.49      3613
   macro avg       0.49      0.49      0.49      3613
weighted avg       0.49      0.49      0.49      3613



In [15]:
print(confusion_matrix(y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))

[[913 941]
 [902 857]]


In [19]:
np.hstack((y_train_conv.reshape(-1,1),km.labels_.reshape(-1,1)))[20:40]

array([[1, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 0],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 1]])